# TITULO 



## Introducción

## Objetivos




## 📚 1. Importación de Librerías y Datos
Importamos las librerías necesarias, configuramos la visualizacion con seaborn de las graficas y damos el directorio donde las guardaremos.

In [3]:
# Librerías para análisis de datos y estadística
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Configuración para guardar figuras
import os
figures_dir = '/Users/miguelrosa/Desktop/RCP Transtelefonica/proyecto/figures'
os.makedirs(figures_dir, exist_ok=True)

print("✅ Librerías importadas correctamente")
print(f"📁 Directorio de figuras: {figures_dir}")

✅ Librerías importadas correctamente
📁 Directorio de figuras: /Users/miguelrosa/Desktop/RCP Transtelefonica/proyecto/figures


>Cargamos el csv con los datos ya limpios, mostramos una preview

In [5]:
# Cargar datos limpios
data_path = '/Users/miguelrosa/Desktop/RCP Transtelefonica/data/cleaned_data.csv'
df = pd.read_csv(data_path)

# Mostrar información básica del dataset
print("📊 INFORMACIÓN DEL DATASET")
print("="*50)
print(f"Total de casos: {len(df)}")
print(f"Variables: {len(df.columns)}")
print(f"\nColumnas disponibles:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

# Vista previa de los datos
print("\n📋 PRIMERAS 5 FILAS:")
print("="*50)
display(df.head())

# Información de tipos de datos y missing values
print("\n🔍 INFORMACIÓN DE VARIABLES:")
print("="*50)
info_df = pd.DataFrame({
    'Tipo': df.dtypes,
    'No Nulos': df.count(),
    'Nulos': df.isnull().sum(),
    '% Nulos': (df.isnull().sum() / len(df) * 100).round(2)
})
display(info_df)

📊 INFORMACIÓN DEL DATASET
Total de casos: 622
Variables: 14

Columnas disponibles:
 1. n_informe
 2. fecha
 3. edad
 4. sexo
 5. rcp_transtelefonica
 6. rcp_testigos
 7. respondiente_rcp
 8. desa_externo
 9. ritmo
10. tiempo_llegada_unidad
11. tiempo_rcp
12. rosc
13. supervivencia_7dias
14. cpc

📋 PRIMERAS 5 FILAS:


,n_informe,fecha,edad,sexo,rcp_transtelefonica,rcp_testigos,respondiente_rcp,desa_externo,ritmo,tiempo_llegada_unidad,tiempo_rcp,rosc,supervivencia_7dias,cpc
0,71090,29/06/2023 18:15,89.0,Masculino,0,0,NaN,0,0,806,1470,1,1,3
1,71934,01/07/2023 13:15,74.0,Masculino,0,0,NaN,0,0,309,180,0,0,5
2,73080,03/07/2023 14:07,58.0,Masculino,1,1,lego,0,1,442,1508,1,1,3
3,73309,04/07/2023 00:43,46.0,Masculino,0,0,NaN,0,0,319,0,0,0,5
4,73865,05/07/2023 13:18,45.0,Masculino,0,0,NaN,1,0,1331,4449,1,0,3



🔍 INFORMACIÓN DE VARIABLES:


,Tipo,No Nulos,Nulos,% Nulos
n_informe,int64,622,0,0.00
fecha,object,622,0,0.00
edad,float64,527,95,15.27
sexo,object,599,23,3.70
rcp_transtelefonica,int64,622,0,0.00
rcp_testigos,int64,622,0,0.00
respondiente_rcp,object,208,414,66.56
desa_externo,int64,622,0,0.00
ritmo,int64,622,0,0.00
tiempo_llegada_unidad,int64,622,0,0.00


Hacemos una verificación rápida de los tipos de datos y valores nulos en el DataFrame. Esto nos ayudará a entender mejor la estructura de los datos y a identificar posibles problemas antes de realizar análisis más profundos.

In [ ]:
# VERIFICACIÓN DE PROBLEMAS EN LOS DATOS
print("\n🔍 VERIFICACIÓN DE CALIDAD DE DATOS")
print("="*60)

# Primero verificar las columnas disponibles
print("Columnas disponibles en el dataset:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

# Mapear nombres de columnas
col_mapping = {}
for col in df.columns:
    col_lower = col.lower().strip()
    if 'sex' in col_lower:
        col_mapping['sexo'] = col
    elif 'rcp_transtelefonica' in col_lower:
        col_mapping['rcp_transtelefonica'] = col
    elif 'rcp_testigos' in col_lower:
        col_mapping['rcp_testigos'] = col
    elif 'respondiente' in col_lower or 'responder' in col_lower:
        col_mapping['respondiente_rcp'] = col

print(f"\nMapeo de columnas identificadas:")
for key, value in col_mapping.items():
    print(f"  {key} -> {value}")

# 1. Verificar distribución de sexo
if 'sexo' in col_mapping:
    print(f"\n1️⃣ DISTRIBUCIÓN POR SEXO:")
    sexo_col = col_mapping['sexo']
    sexo_counts = df[sexo_col].value_counts()
    sexo_pct = df[sexo_col].value_counts(normalize=True) * 100
    for sexo, count in sexo_counts.items():
        pct = sexo_pct[sexo]
        print(f"   • {sexo}: {count} casos ({pct:.1f}%)")
    
    # Verificar si hay problemas con la distribución del sexo
    masculino_pct = 0
    for sexo in sexo_counts.index:
        if 'masc' in str(sexo).lower() or 'male' in str(sexo).lower() or 'h' == str(sexo).lower() or 'm' == str(sexo).lower():
            masculino_pct += sexo_pct[sexo]
    
    if masculino_pct < 60:
        print(f"   ⚠️ POSIBLE PROBLEMA: Solo {masculino_pct:.1f}% son masculinos (esperado ~70%)")
    else:
        print(f"   ✅ Distribución de sexo parece correcta: {masculino_pct:.1f}% masculinos")

# 2. Verificar relación RCP transtelefónica vs RCP testigos
if 'rcp_transtelefonica' in col_mapping and 'rcp_testigos' in col_mapping:
    print(f"\n2️⃣ RELACIÓN RCP TRANSTELEFÓNICA vs RCP TESTIGOS:")
    rcp_trans_col = col_mapping['rcp_transtelefonica']
    rcp_test_col = col_mapping['rcp_testigos']
    
    tabla_rcp_relacion = pd.crosstab(df[rcp_trans_col], df[rcp_test_col], margins=True)
    print("   Tabla de contingencia (filas=RCP Trans, columnas=RCP Testigos):")
    display(tabla_rcp_relacion)
    
    # Casos problemáticos: RCP transtelefónica SIN testigos
    casos_problema = df[(df[rcp_trans_col] == 1) & (df[rcp_test_col] == 0)]
    print(f"\n   ⚠️ CASOS PROBLEMÁTICOS (RCP Trans=1, RCP Testigos=0): {len(casos_problema)}")
    
    if len(casos_problema) > 0:
        print("   Estos casos son teóricamente imposibles (no puede haber RCP transtelefónica sin testigos)")
        print("   Primeros 5 casos problemáticos:")
        cols_to_show = ['n_informe', rcp_trans_col, rcp_test_col]
        if 'respondiente_rcp' in col_mapping:
            cols_to_show.append(col_mapping['respondiente_rcp'])
        display(casos_problema[cols_to_show].head())
    else:
        print("   ✅ No hay casos problemáticos")

# 3. Análisis de respondiente_rcp
if 'respondiente_rcp' in col_mapping:
    print(f"\n3️⃣ ANÁLISIS DE TIPO DE RESPONDIENTE:")
    resp_col = col_mapping['respondiente_rcp']
    respondiente_counts = df[resp_col].value_counts()
    print("   Distribución de respondientes:")
    for resp, count in respondiente_counts.items():
        pct = count / len(df) * 100
        print(f"   • '{resp}': {count} casos ({pct:.1f}%)")

# 4. Casos con RCP transtelefónica
if 'rcp_transtelefonica' in col_mapping:
    rcp_trans_col = col_mapping['rcp_transtelefonica']
    casos_rcp_trans = df[df[rcp_trans_col] == 1]
    print(f"\n4️⃣ CASOS CON RCP TRANSTELEFÓNICA:")
    print(f"   • Total: {len(casos_rcp_trans)} casos ({len(casos_rcp_trans)/len(df)*100:.1f}%)")
    
    if 'rcp_testigos' in col_mapping:
        rcp_test_col = col_mapping['rcp_testigos']
        print(f"   • Con testigos: {casos_rcp_trans[rcp_test_col].sum()}")
        print(f"   • Sin testigos: {len(casos_rcp_trans) - casos_rcp_trans[rcp_test_col].sum()}")

print(f"\n📋 RECOMENDACIONES PARA CORRECCIÓN:")
if 'rcp_transtelefonica' in col_mapping and 'rcp_testigos' in col_mapping:
    casos_problema = df[(df[col_mapping['rcp_transtelefonica']] == 1) & (df[col_mapping['rcp_testigos']] == 0)]
    if len(casos_problema) > 0:
        print(f"   1. ❌ ELIMINAR análisis de RCP transtelefónica vs RCP testigos (lógicamente inconsistente)")
        print(f"   2. Los {len(casos_problema)} casos con RCP Trans=1 y Testigos=0 necesitan revisión")
        
print(f"   3. ✅ ENFOCAR análisis en: RCP transtelefónica vs outcomes (ROSC, supervivencia, CPC)")
print(f"   4. ✅ ANALIZAR tiempo de llegada como variable de confusión")
print(f"   5. ✅ ESTRATIFICAR por edad, sexo, ritmo inicial como variables importantes")


🔍 VERIFICACIÓN DE CALIDAD DE DATOS
Columnas disponibles en el dataset:
 1. n_informe
 2. fecha
 3. edad
 4. sexo
 5. rcp_transtelefonica
 6. rcp_testigos
 7. respondiente_rcp
 8. desa_externo
 9. ritmo
10. tiempo_llegada_unidad
11. tiempo_rcp
12. rosc
13. supervivencia_7dias
14. cpc

Mapeo de columnas identificadas:
  sexo -> sexo
  rcp_transtelefonica -> rcp_transtelefonica
  rcp_testigos -> rcp_testigos
  respondiente_rcp -> respondiente_rcp

1️⃣ DISTRIBUCIÓN POR SEXO:
   • Masculino: 463 casos (77.3%)
   • Femenino: 136 casos (22.7%)
   ✅ Distribución de sexo parece correcta: 77.3% masculinos

2️⃣ RELACIÓN RCP TRANSTELEFÓNICA vs RCP TESTIGOS:
   Tabla de contingencia (filas=RCP Trans, columnas=RCP Testigos):


rcp_testigos,0,1,All
rcp_transtelefonica,,,
0,414,146,560
1,0,62,62
All,414,208,622



   ⚠️ CASOS PROBLEMÁTICOS (RCP Trans=1, RCP Testigos=0): 0
   ✅ No hay casos problemáticos

3️⃣ ANÁLISIS DE TIPO DE RESPONDIENTE:
   Distribución de respondientes:
   • 'lego': 122 casos (19.6%)
   • 'sanitario': 74 casos (11.9%)
   • 'policia': 10 casos (1.6%)
   • 'bombero': 2 casos (0.3%)

4️⃣ CASOS CON RCP TRANSTELEFÓNICA:
   • Total: 62 casos (10.0%)
   • Con testigos: 62
   • Sin testigos: 0

📋 RECOMENDACIONES PARA CORRECCIÓN:
   3. ✅ ENFOCAR análisis en: RCP transtelefónica vs outcomes (ROSC, supervivencia, CPC)
   4. ✅ ANALIZAR tiempo de llegada como variable de confusión
   5. ✅ ESTRATIFICAR por edad, sexo, ritmo inicial como variables importantes


## Background. Datos
SAMUR-PC entre 2023 y 2025...





In [ ]:
# AQUI PONEMOS LOS HISTOGRAMAS Y TABLAS, DESCRIBIMOS LOS DATOS LIMPIADOS

## Metodología

Xi cuadrado, t student, se compara, etc....

## Resultados

## Conclusiones